<a href="https://colab.research.google.com/github/Saibhossain/lungCancer_survival_model_analysis/blob/main/CNN_VS_Vit_lung_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## setup

In [1]:
!pip install torch torchvision torchaudio tqdm scikit-learn
!pip install torchio
!pip install numpy
!pip install torchtuples pycox
!pip install lifelines
!pip install SimpleITK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.9/193.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 20.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.9/429.9 kB 26.5 MB/s eta 0:00:00
  Created wheel for feather-format: filename=feather_format-0.4.1-py3-none-any.whl size=2434 sha256=03cba687f5dc675edd3eaac54c13a6526ba14faa456ba4bf2b024a4616a37e26

In [2]:
!pip install git+https://github.com/AIM-Harvard/pyradiomics.git --no-cache-dir --use-pep517
!pip install pyradiomics
!pip install dcmrtstruct2nii

  Cloning https://github.com/AIM-Harvard/pyradiomics.git to /tmp/pip-req-build-u6iuf_a2
  Running command git clone --filter=blob:none --quiet https://github.com/AIM-Harvard/pyradiomics.git /tmp/pip-req-build-u6iuf_a2
  Resolved https://github.com/AIM-Harvard/pyradiomics.git to commit 8ed579383b44806651c463d5e691f3b2b57522ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.2/788.2 kB 68.6 MB/s eta 0:00:00
  Created wheel for pyradiomics: filename=pyradiomics-3.1.1.dev111+g8ed579383-cp312-cp312-linux_x86_64.whl size=121811 sha256=750d5de8d5327574977553e9cb05fdfaff68b4c26ae3f417fea4ad583b75f09d
  Store

In [3]:
!pip install shap
!pip install rt-utils scikit-survival

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.0/300.0 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.1/222.1 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 71.3 MB/s eta 0:00:00
  Attempting uninstall: osqp
    Found existing installation: osqp 1.0.5
    Uninstalling osqp-1.0.5:
      Successfully uninstalled osqp-1.0.5


## Dataset

In [4]:
import kagglehub

path = kagglehub.dataset_download("umutkrdrms/nsclc-radiomics")

print("Path to dataset files:", path)

100%|██████████| 11.7G/11.7G [02:37<00:00, 79.8MB/s]


Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/umutkrdrms/nsclc-radiomics/versions/2


### datapath

In [5]:

from pathlib import Path
import pandas as pd

DATA_ROOT = Path("/root/.cache/kagglehub/datasets/umutkrdrms/nsclc-radiomics/versions/2/NSCLC-Radiomics")
OUTPUT_CSV = "ct_seg_paths.csv"
MIN_CT_FILES = 10
records = []

for study_dir in sorted(DATA_ROOT.glob("LUNG1-*/*")):
    if not study_dir.is_dir():
        continue

    patient_id = study_dir.parent.name
    seg_files = []
    for item in study_dir.iterdir():
        if item.is_dir() and item.name.startswith("300."):
            seg_dcms = list(item.rglob("*.dcm"))
            seg_files.extend(seg_dcms)

    seg_file_path = str(seg_files[0]) if seg_files else "NA"
    best_ct_dir = None
    max_ct_count = 0
    rat_file_path = None

    for item in study_dir.iterdir():
        if not (item.is_dir() and not item.name.startswith("300.")):
            continue

        ct_dcms = list(item.rglob("*.dcm"))
        if len(ct_dcms) >= MIN_CT_FILES and len(ct_dcms) > max_ct_count:
            max_ct_count = len(ct_dcms)
            best_ct_dir = item
        if len(ct_dcms) == 1 and rat_file_path is None:
            rat_file_path = str(ct_dcms[0])

    if best_ct_dir is None:
        print(f" No valid CT folder (≥{MIN_CT_FILES} files) for {patient_id}")
        continue

    records.append({
        "patient_id": patient_id,
        "study_folder": study_dir.name,
        "ct_folder_path": str(best_ct_dir),
        "seg_file_path": seg_file_path,
        "rat_file_path": rat_file_path
    })

df = pd.DataFrame(records)
df.to_csv(OUTPUT_CSV, index=False)

print(f"\n Saved {len(df)} patient records to '{OUTPUT_CSV}'")
print("\nFirst few rows:")
print(df.head())


 Saved 422 patient records to 'ct_seg_paths.csv'

First few rows:
  patient_id                 study_folder  \
0  LUNG1-001  09-18-2008-StudyID-NA-69331   
1  LUNG1-002  01-01-2014-StudyID-NA-85095   
2  LUNG1-003  01-01-2014-StudyID-NA-34270   
3  LUNG1-004  09-24-2006-StudyID-NA-27873   
4  LUNG1-005  01-01-2014-StudyID-NA-93819   

                                      ct_folder_path  \
0  /root/.cache/kagglehub/datasets/umutkrdrms/nsc...   
1  /root/.cache/kagglehub/datasets/umutkrdrms/nsc...   
2  /root/.cache/kagglehub/datasets/umutkrdrms/nsc...   
3  /root/.cache/kagglehub/datasets/umutkrdrms/nsc...   
4  /root/.cache/kagglehub/datasets/umutkrdrms/nsc...   

                                       seg_file_path  \
0  /root/.cache/kagglehub/datasets/umutkrdrms/nsc...   
1  /root/.cache/kagglehub/datasets/umutkrdrms/nsc...   
2  /root/.cache/kagglehub/datasets/umutkrdrms/nsc...   
3  /root/.cache/kagglehub/datasets/umutkrdrms/nsc...   
4  /root/.cache/kagglehub/datasets/umutkrdrms

# Feature Extraction

In [ ]:
import os
import numpy as np
import pandas as pd
import pydicom
import SimpleITK as sitk
from pathlib import Path
from rt_utils import RTStructBuilder
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ----------------------------
# 1. Model Setup (ResNet & ViT)
# ----------------------------
def get_models():
    # --- ResNet50 ---
    # We remove the last classification layer (fc) to get the raw features (2048 dim)
    resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
    resnet.fc = nn.Identity()
    resnet.to(device)
    resnet.eval()

    # --- ViT Base 16 ---
    # We remove the heads to get the representation (768 dim)
    vit = models.vit_b_16(weights=models.ViT_B_16_Weights.DEFAULT)
    vit.heads = nn.Identity()
    vit.to(device)
    vit.eval()

    return resnet, vit

# Define standard ImageNet transforms
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ----------------------------
# 2. Medical Image Helper Functions
# ----------------------------

def apply_lung_window(np_image, wl=-600, ww=1500):
    """
    Converts CT Hounsfield Units to 0-255 grayscale using a Lung Window.
    Lung Window: Center (WL) -600, Width (WW) 1500.
    """
    lower_bound = wl - (ww / 2)
    upper_bound = wl + (ww / 2)

    # Clip and normalize to 0-1
    img_clipped = np.clip(np_image, lower_bound, upper_bound)
    img_norm = (img_clipped - lower_bound) / (upper_bound - lower_bound)

    # Convert to 0-255 uint8
    return (img_norm * 255).astype(np.uint8)

def get_largest_tumor_slice(ct_sitk, mask_sitk):
    """
    Finds the slice index with the largest tumor area.
    Returns: The 2D slice (H, W) converted to 3-channel RGB.
    """
    ct_arr = sitk.GetArrayFromImage(ct_sitk)   # (D, H, W)
    mask_arr = sitk.GetArrayFromImage(mask_sitk) # (D, H, W)

    # Calculate tumor area per slice
    tumor_areas = np.sum(mask_arr, axis=(1, 2))

    if np.max(tumor_areas) == 0:
        raise ValueError("Mask is empty (no tumor found).")

    # Get index of slice with largest tumor
    max_slice_idx = np.argmax(tumor_areas)

    # Extract that slice
    best_slice = ct_arr[max_slice_idx]

    # Apply Lung Window (Important for Lung Cancer!)
    img_windowed = apply_lung_window(best_slice)

    # Convert to PIL Image and make it 3-channel (Fake RGB)
    # ResNet/ViT expect 3 channels. We duplicate the grayscale channel 3 times.
    pil_img = Image.fromarray(img_windowed).convert("RGB")

    return pil_img

# ----------------------------
# 3. Existing Loading Helpers (Your Code)
# ----------------------------
def load_ct_as_sitk(ct_folder):
    reader = sitk.ImageSeriesReader()
    files = reader.GetGDCMSeriesFileNames(str(ct_folder))
    if not files:
        raise FileNotFoundError(f"No DICOM files found in {ct_folder}")
    reader.SetFileNames(files)
    return reader.Execute()

def extract_tumor_mask_from_seg(seg_file, ct_img, ct_folder):
    # (Simplified version of your code for brevity, assumes your logic works)
    seg_ds = pydicom.dcmread(str(seg_file), force=True)
    dcm_files = [f for f in Path(ct_folder).rglob("*.dcm") if f.is_file()]
    # ... [Insert your specific SEG sorting/parsing logic here if needed] ...
    # For this script, I assume you use your existing function here.
    # To make this runnable, I'll mock the return if you don't paste your full function back in.
    # **Please ensure your original `extract_tumor_mask_from_seg` is pasted here.**
    pass

def extract_tumor_mask_from_rtstruct(rtstruct_file, ct_folder):
    rtstruct = RTStructBuilder.create_from(
        dicom_series_path=str(ct_folder),
        rt_struct_path=str(rtstruct_file)
    )
    roi_names = rtstruct.get_roi_names()
    target_roi = None
    for name in roi_names:
        if any(kw in name for kw in ["GTV-1", "GTV", "Tumor", "Primary"]):
            target_roi = name
            break
    if not target_roi:
        target_roi = roi_names[0]

    mask_3d = rtstruct.get_roi_mask_by_name(target_roi)
    mask_array = np.transpose(mask_3d.astype(np.uint8), (2, 0, 1))
    ct_img = load_ct_as_sitk(ct_folder)
    mask_sitk = sitk.GetImageFromArray(mask_array)
    mask_sitk.CopyInformation(ct_img)
    return mask_sitk

# ----------------------------
# 4. Main Execution Loop
# ----------------------------

# Initialize Models
resnet_model, vit_model = get_models()

# Load Data List
csv_path = "/content/ct_seg_paths.csv"
df = pd.read_csv(csv_path).replace("NA", np.nan)

dl_results = []
print(f"Starting Deep Feature Extraction for {len(df)} patients...")

for i, row in df.iterrows():
    patient_id = row["patient_id"]
    ct_folder = Path(row["ct_folder_path"])
    seg_file = row["seg_file_path"]
    rtstruct_file = Path(row["rat_file_path"])

    if i % 10 == 0: print(f"Processing {i}/{len(df)}...")

    try:
        ct_img = load_ct_as_sitk(ct_folder)
        mask_sitk = None

        # --- Strategy: Try SEG, then RTSTRUCT ---
        if pd.notna(seg_file) and Path(seg_file).is_file():
            try:
                # Use your original SEG logic function here
                # mask_sitk = extract_tumor_mask_from_seg(seg_file, ct_img, ct_folder)
                pass # (Placeholder: Ensure your function is defined above)
            except: pass

        if mask_sitk is None:
            try:
                mask_sitk = extract_tumor_mask_from_rtstruct(rtstruct_file, ct_folder)
            except: pass

        if mask_sitk is None:
            print(f"Skipping {patient_id}: No valid mask found.")
            continue

        # --- PREPARE IMAGE FOR DL ---
        # 1. Get the 2D slice with the biggest tumor
        pil_img = get_largest_tumor_slice(ct_img, mask_sitk)

        # 2. Transform to Tensor
        input_tensor = preprocess(pil_img).unsqueeze(0).to(device) # Shape: (1, 3, 224, 224)

        # --- EXTRACT FEATURES ---
        with torch.no_grad():
            # ResNet Features (2048 dims)
            resnet_feats = resnet_model(input_tensor).cpu().numpy().flatten()

            # ViT Features (768 dims)
            vit_feats = vit_model(input_tensor).cpu().numpy().flatten()

        # Save to dict
        row_data = {"patient_id": patient_id}

        # Add ResNet columns
        for idx, val in enumerate(resnet_feats):
            row_data[f"ResNet_{idx}"] = val

        # Add ViT columns
        for idx, val in enumerate(vit_feats):
            row_data[f"ViT_{idx}"] = val

        dl_results.append(row_data)

    except Exception as e:
        print(f"Error on {patient_id}: {e}")

# ----------------------------
# 5. Save Results
# ----------------------------
if dl_results:
    # Note: This DataFrame will be WIDE (2048 + 768 columns)
    feat_df = pd.DataFrame(dl_results)
    output_file = "deep_learning_features_resnet_vit.csv"
    feat_df.to_csv(output_file, index=False)
    print(f" Success! Saved deep features to {output_file}")
    print(f"Shape: {feat_df.shape}")
else:
    print(" No features extracted.")

Using device: cuda
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 192MB/s]


Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth


100%|██████████| 330M/330M [00:02<00:00, 156MB/s]


Starting Deep Feature Extraction for 422 patients...
Processing 0/422...
Processing 10/422...
Processing 20/422...
Processing 30/422...
Skipping LUNG1-035: No valid mask found.
Processing 40/422...
Processing 50/422...
Processing 60/422...
Processing 70/422...
Processing 80/422...
Processing 90/422...
Processing 100/422...
Processing 110/422...
Processing 120/422...
Processing 130/422...
Processing 140/422...
Processing 150/422...
Processing 160/422...
Processing 170/422...
Processing 180/422...
Processing 190/422...
Processing 200/422...
Processing 210/422...
Processing 220/422...
Processing 230/422...
Processing 240/422...
Processing 250/422...
Processing 260/422...
Processing 270/422...
Processing 280/422...
Processing 290/422...
Processing 300/422...
Processing 310/422...
Processing 320/422...
Processing 330/422...
Processing 340/422...
